In [49]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers
from tensorflow.keras.layers import Lambda,Permute,Multiply
from tensorflow.keras.layers import Input,Dense,Flatten,Softmax,Embedding,Reshape,SimpleRNN,GRU,LSTM,Bidirectional,Dropout,TimeDistributed
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

tf.__version__, K.__version__

('1.9.0', '2.1.6-tf')

In [26]:
tf.reset_default_graph()
# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)  # vocab list

# Parameter
n_step = 5  # maxium number of words in one sentence(=number of time steps)
n_hidden = 128

def make_batch(sentences):
    input_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[0].split()]]]
    output_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[1].split()]]]
    target_batch = [[word_dict[n] for n in sentences[2].split()]]
    return np.array(input_batch), np.array(output_batch), np.array(target_batch).reshape(1,-1,1)

input_batch, output_batch, target_batch = make_batch(sentences)

In [27]:
target_batch.shape

(1, 5, 1)

# Keras 自定义层示意

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # 为该层创建一个可训练的权重
#         self.kernel = self.add_weight(name='kernel', 
#                                       shape=(input_shape[1], self.output_dim),
#                                       initializer='uniform',
#                                       trainable=True)
        self.kernel = self.add_weight(name='atteention_weight', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # 一定要在最后调用它

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

# Attention layer

In [76]:
def attention_3d_block(inputs, single_attention_vector=False):
    # 如果上一层是LSTM，需要return_sequences=True
    # inputs.shape = (batch_size, time_steps, input_dim) = (?, 5, 128)
    time_steps = np.shape(inputs)[1]
    input_dim = np.shape(inputs)[2]
    a = Permute((2, 1))(inputs) # shape = (?, 128, 5)
    a = Dense(time_steps, activation='softmax')(a) # shape = (?, 128, 5)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)

    a_probs = Permute((2, 1))(a) # shape = (?, 5, 128)
    print('a_probs维度：',np.shape(a_probs))
    # 乘上了attention权重，但是并没有求和，好像影响不大
    # 如果分类任务，进行Flatten展开就可以了
    # element-wise
    output_attention_mul = Multiply()([inputs, a_probs]) # shape = (?, 5, 128)
    return output_attention_mul

In [64]:
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(** kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        # W.shape = (time_steps, time_steps)
        self.W = self.add_weight(name='att_weight', 
                                 shape=(input_shape[1], input_shape[1]),
                                 initializer='uniform',
                                 trainable=True)
        self.b = self.add_weight(name='att_bias', 
                                 shape=(input_shape[1],),
                                 initializer='uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # inputs.shape = (batch_size, time_steps, seq_len)
        x = K.permute_dimensions(inputs, (0, 2, 1))
        # x.shape = (batch_size, seq_len, time_steps)
        a = K.softmax(K.tanh(K.dot(x, self.W) + self.b))
        outputs = K.permute_dimensions(a * x, (0, 2, 1))
        outputs = K.sum(outputs, axis=1)
        return outputs

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

In [37]:
# Attention GRU network       
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [74]:
encoder_input = Input(shape=(n_step, n_class)) # [batch_size, n_step, n_lcass]
decoder_input = Input(shape=(n_step, n_class))

encoder = LSTM(n_hidden, return_sequences=True, return_state=True)
# dropout
encoder_output, state_h, state_c = encoder(encoder_input)
encoder_state = [state_h, state_c]

decoder = LSTM(n_hidden, return_sequences=True)
decoder_output = decoder(decoder_input, initial_state = encoder_state)

# attention = attention_3d_block(())
attention_output = attention_3d_block(decoder_output)

dense = Dense(n_class, activation='softmax')
outputs = dense(attention_output)

model = Model([encoder_input, decoder_input], outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit([input_batch, output_batch], target_batch, epochs=100)

init： (?, 5, 128)
第一次维度： (?, 128, 5)
Dense后维度： (?, 128, 5)
a_probs维度： (?, 5, 128)
权重维度： (?, 5, 128)
输出维度： (?, 5, 128)
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.3958 - acc: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 2.3801 - acc: 0.6000
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 2.3662 - acc: 0.6000
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 2.3498 - acc: 0.6000
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 2.3272 - acc: 0.6000
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 2.2922 - acc: 0.8000
Epoch 7/100
1/1 [==============================] - 0s 19ms/step - loss: 2.2330 - acc: 0.8000
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 2.1318 - acc: 0.6000
Epoch 9/100
1/1 [==============================] - 0s 18ms/step - loss: 1.9872 - acc: 0.6000
Epoch 10/100
1/1 [=========================

1/1 [==============================] - 0s 8ms/step - loss: 0.4366 - acc: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 14ms/step - loss: 0.4255 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4224 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4077 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 15ms/step - loss: 0.3999 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 10ms/step - loss: 0.3902 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 12ms/step - loss: 0.3843 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 0s 13ms/step - loss: 0.3775 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 0s 14ms/step - loss: 0.3750 - acc: 1.0000
Epoch 96/100
1/1 [==============================] - 0s 15ms/step - loss: 0.3675 - acc: 1.0000
Epoch 97/100
1/1 [==============================] - 0s 14ms/step - loss: 0

In [63]:
predict_batch = [np.eye(n_class)[[word_dict[n] for n in 'P P P P P'.split()]]]
result = model.predict([input_batch, np.array(predict_batch)])
result = np.argmax(result, axis=2)
print(sentences[0].split(), '->', [number_dict[n] for n in result[0]])

['ich', 'mochte', 'ein', 'bier', 'P'] -> ['i', 'want', 'a', 'beer', 'E']
